# **SharkBot** - NLP para "O Guarani" de José de Alencar

**INTEGRANTES:**\
Guilherme Henrique Eduardo de Lara Peres\
Henrique Anderle Schulz\
Lucas Azevedo Dias\
Rafaela de Miranda


---

**Obs.: Talvez o Colab peça para reiniciar o ambiente pela troca da versão do NumPy.**

# **Install & Imports**


In [1]:
!pip install numpy==1.25.0 gensim==4.3.3 transformers==4.46.3

In [2]:
import collections
import copy
import datetime
import functools
import gensim
import gzip
import IPython.display
import ipywidgets as widgets
import langchain.text_splitter
import nltk
import nltk.corpus
import nltk.tokenize
import nltk.tokenize.treebank
import numpy as np
import os
import pandas as pd
import re
import regex
import shutil
import sklearn.metrics.pairwise
import spacy
import string
import time
import torch
import transformers
import urllib.request
import zoneinfo

In [3]:
!python -m spacy download pt_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 12.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

# **Carregando arquivos**

## Criando diretórios

In [5]:
%cd /content

!mkdir -p NLP
%cd /content/NLP

!mkdir -p processed

/content
/content/NLP


## Arquivo do livro "O Guarani"

In [6]:
input_filepath = "o_guarani.txt"
processed_input_filepath = f"./processed/{input_filepath}"

### Realizando *download*

In [7]:
# URL do arquivo
url = "https://archive.org/download/bub_gb_RZFcAAAAQBAJ/bub_gb_RZFcAAAAQBAJ_hocr_searchtext.txt.gz"
gz_filename = "bub_gb_RZFcAAAAQBAJ_hocr_searchtext.txt.gz"
output_filename = input_filepath

# Baixa o arquivo .gz
urllib.request.urlretrieve(url, gz_filename)

# Descompacta o arquivo na mesma pasta
with gzip.open(gz_filename, 'rb') as f_in:
    with open(output_filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Remove o arquivo temporário
os.remove(gz_filename)

### Limpando o arquivo



In [8]:
start_line = 96
end_line = 5549

In [9]:
with open(input_filepath, encoding='utf-8') as infile, open(processed_input_filepath, 'w', encoding='utf-8') as outfile:
    for lineno, line in enumerate(infile, start=1):
        if lineno < start_line:
            continue
        if lineno > end_line:
            break
        if line.strip() == '':
            continue
        outfile.write(line)

with open(processed_input_filepath, encoding='utf-8') as outfile:
    text = outfile.read()

with open(processed_input_filepath, "w", encoding='utf-8') as outfile:
    text = regex.sub(r'(?<=\p{L})-\p{Zs}(?=\p{L})', '-', text) # Corrige defeito de escrita como em "torna- se"
    text = re.sub(r'\s{2,}', ' ', text) # Remove espaços extras
    text = re.sub(r'(\n)?Copyrigh[ti]ed material(\n)?', ' ', text)
    text = re.sub(r'PARTE .*\n.*\n', '\n', text) # Remove títulos de partes
    text = re.sub(r'\n• • •\n', '\n', text)
    capitulos = [m.split("\n")[:2] for m in regex.findall(r'(?:\d*)\n(?:\p{Lu}|\p{Zs}|\p{P})+\n', text) if m]
    for number, capitulo in capitulos:
        text = re.sub(f'{number}\n{capitulo}\n', f'\n', text) # Remove números e títulos de capítulos
    text = re.sub(r' \n', '\n', text) # Remove espaço seguido \n
    text = re.sub(r'\n{1,}', '\n', text) # Remove \n extras
    text = re.sub(r'^\n', '', text) # Remove \n inicial se houver
    text = text.replace('-\n', '-') # Endereça problema encontrado de separar "iluminou-se" em duas linhas
    # text = re.sub(r'—', '', text) # Remove travessoes
    # text = re.sub(r'\n{1,}', ' ', text) # Remove \n
    # text = regex.sub("(?<=\p{L})-(?=\p{L})", " ", text) # Remove hífen entre letras
    outfile.write(text)

In [10]:
print(len(capitulos), capitulos)

54 [['', 'CENÁRIO'], ['2', 'LEALDADE'], ['3', 'A BANDEIRA'], ['4', 'CAÇADA'], ['5', 'LOURA E MORENA'], ['6', 'A VOLTA'], ['7', 'APRECE'], ['8', 'TRES LINHAS'], ['9', 'AMOR'], ['10', 'AO ALVORECER'], ['11', 'NO BANHO'], ['12', 'A ONÇA'], ['13', 'REVELAÇÃO'], ['14', 'A ÍNDIA'], ['15', 'OSTRES'], ['', 'O CARMELITA'], ['17', 'IARA!'], ['18', 'GÉNIO DO MAL'], ['19', 'CECI'], ['20', 'VILANIA'], ['21', 'NOBREZA'], ['22', 'NO PRECIPÍCIO'], ['23', 'O BRACELETE'], ['24', 'TESTAMENTO'], ['25', 'DESPEDIDA'], ['26', 'TRAVESSURA'], ['27', 'PELO AR'], ['28', 'TRAMA'], ['29', 'A XÁCARA'], ['', 'PARTIDA'], ['31', 'PREPARATIVOS'], ['32', 'VERME E FLOR'], ['33', 'NA TREVA'], ['34', 'DEUS DISPÕE'], ['35', 'REVOLTA'], ['36', 'OS SELVAGENS'], ['37', 'DESÂNIMO'], ['38', 'ESPERANÇA'], ['39', 'NA BRECHA'], ['40', 'O FRADE'], ['41', 'DESOBEDIÊNCIA'], ['42', 'COMBATE'], ['43', 'O PRISIONEIRO'], ['', 'ARREPENDIMENTO'], ['45', 'O SACRIFÍCIO'], ['46', 'SORTIDA'], ['47', 'REVELAÇÃO'], ['48', 'O PAIOL'], ['49', 'TRÉG

# **Tratando o texto**

##Carregando texto...

In [11]:
# Carregando texto...
sample = open(processed_input_filepath, encoding='utf-8')
text = sample.read()
text

'De um dos cabeços da Serra dos Órgãos desliza um fio de água que se dirige para o norte, e engrossado com os mananciais que recebe no seu curso de dez léguas, torna-se rio caudal.\nÉ o Paquequer: saltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espreguiçar na várzea e embeber no Paraíba, que rola majestosamente em seu vasto leito.\nDir-se-ia que, vassalo e tributário desse rei das águas, o pequeno rio, altivo e sobranceiro contra os rochedos, curva-se humildemente aos pés do suserano. Perde então a beleza selvática; suas ondas são calmas e serenas como as de um lago, e não se revoltam contra os barcos e as canoas que resvalam sobre elas: escravo submisso, sofre o látego do senhor.\nNão é neste lugar que ele deve ser visto; sim três ou quatro léguas acima de sua foz, onde é livre ainda, como o filho indómito desta pátria da liberdade.\nAí, o Paquequer lança-se rápido sobre o seu leito, e atravessa as florestas como o tapir, espumando, deixando o pelo espar

### Ajuste de abreviações

In [12]:
# Ajusta as abreviações no texto
def fix_abbreviations(text):
    # Dom ou dona
    text = re.sub(r"(?:\s)(D|d)(\.)? ", " ", text)

    # Senhor
    text = re.sub(r"(?:\s)(Sr|sr)(\.)? ", " ", text)

    # Frade
    text = re.sub(r"(?:\s)(Fr|fr)(\.)? ", "frade ", text)

    # Doutor
    text = re.sub(r"(?:\s)(Dr|dr)(\.)? ", "doutor ", text)

    return text

In [13]:
text = fix_abbreviations(text)
text

'De um dos cabeços da Serra dos Órgãos desliza um fio de água que se dirige para o norte, e engrossado com os mananciais que recebe no seu curso de dez léguas, torna-se rio caudal.\nÉ o Paquequer: saltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espreguiçar na várzea e embeber no Paraíba, que rola majestosamente em seu vasto leito.\nDir-se-ia que, vassalo e tributário desse rei das águas, o pequeno rio, altivo e sobranceiro contra os rochedos, curva-se humildemente aos pés do suserano. Perde então a beleza selvática; suas ondas são calmas e serenas como as de um lago, e não se revoltam contra os barcos e as canoas que resvalam sobre elas: escravo submisso, sofre o látego do senhor.\nNão é neste lugar que ele deve ser visto; sim três ou quatro léguas acima de sua foz, onde é livre ainda, como o filho indómito desta pátria da liberdade.\nAí, o Paquequer lança-se rápido sobre o seu leito, e atravessa as florestas como o tapir, espumando, deixando o pelo espar

##Processmento com Spacy

In [14]:
# Carrega modelo spaCy
nlp = spacy.load("pt_core_news_md")

In [15]:
# Processa o texto com spaCy
doc = nlp(text)

In [16]:
cols = ("text", "lemma", "pos", "tag", "dep", "form", "is_alpha", "is_stop")
rows = []
for token in doc:
    row = [token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)

df

,text,lemma,pos,tag,dep,form,is_alpha,is_stop
0,De,de,ADP,ADP,case,Xx,True,True
1,um,um,NUM,NUM,obl,xx,True,True
2,dos,de o,ADP,ADP,case,xxx,True,True
3,cabeços,cabeço,NOUN,NOUN,nmod,xxxx,True,False
4,da,de o,ADP,ADP,case,xx,True,True
...,...,...,...,...,...,...,...,...
127618,sumiu-se,sumiur se,VERB,VERB,ROOT,xxxx-xx,False,False
127619,no,em o,ADP,ADP,case,xx,True,True
127620,horizonte,horizonte,NOUN,NOUN,obl,xxxx,True,False
127621,.,.,PUNCT,PUNCT,punct,.,False,False


## Tokens e sentenças

In [17]:
all_tokens = [token for token in doc if not token.is_stop]

all_tokens[:10]

[cabeços, Serra, Órgãos, desliza, fio, água, dirige, norte, ,, engrossado]

In [18]:
all_sentences = [sent.text for sent in doc.sents]

print(all_sentences[:1])

['De um dos cabeços da Serra dos Órgãos desliza um fio de água que se dirige para o norte, e engrossado com os mananciais que recebe no seu curso de dez léguas, torna-se rio caudal.\n']


In [19]:
all_sentences_tokens = [
    [
        token
        for token in sent
        if not token.is_stop and not token.is_punct and token.pos_ in ("NOUN", "PROPN", "VERB", "ADJ", "ADV")
    ]
    for sent in doc.sents
]

print(all_sentences_tokens[:1])

[[cabeços, Serra, Órgãos, desliza, fio, água, dirige, norte, engrossado, mananciais, recebe, curso, léguas, torna-se, rio, caudal]]


## Definindo tokens de interesse

In [20]:
# Formata lematização
def format_lemma(lemma):
    return lemma.split()[0].split("-")[0].lower()

### Definindo entidades com NER aplicado com transformers

In [21]:
# Modelo para NER
# https://dataloop.ai/library/model/lfcc_bert-portuguese-ner/

model_name = "lfcc/bert-portuguese-ner"
ner_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
ner_model = transformers.BertForTokenClassification.from_pretrained(model_name)

ner = transformers.pipeline(
    "ner",
    model=ner_model,
    tokenizer=ner_tokenizer,
    aggregation_strategy="average",
    device=(0 if torch.cuda.is_available() else -1)
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [22]:
# Retorna todas as entidades encontradas nas sequências de texto
def get_ner_entities(sentences):
    sentences_formatted = []
    detok = nltk.tokenize.TreebankWordDetokenizer()
    for sentence in sentences:
        sentences_formatted.append(list(nltk.tokenize.word_tokenize(sentence, language='portuguese')))
        sentences_formatted[-1] = [token.strip('_').replace('--', '—').replace('`',"'").replace("''",'"') for token in sentences_formatted[-1]]
        sentences_formatted[-1] = detok.detokenize(sentences_formatted[-1])

    ner_entities = [ent for result in ner(sentences) for ent in result]

    # Obter todos os entity_group disponíveis
    all_entity_groups = {ent['entity_group'] for ent in ner_entities}

    # Inicializar categorias dinamicamente
    categorias_final = {group.lower(): list() for group in all_entity_groups}

    # Criar um contador de frequências global
    frequencia_global = collections.defaultdict(dict)

    # Contar ocorrências por categoria
    contadores = {}
    for group in all_entity_groups:
        palavras = [ent['word'] for ent in ner_entities if ent['entity_group'] == group]
        contadores[group.lower()] = collections.Counter(palavras)

    # Popular frequencia_global
    for categoria, contador in contadores.items():
        for palavra, freq in contador.items():
            palavra_lower = palavra.lower()
            frequencia_global[palavra_lower][categoria] = freq

    # Determinar a categoria mais frequente de cada palavra
    for palavra, categorias in frequencia_global.items():
        categoria_mais_frequente = max(categorias.items(), key=lambda item: item[1])[0]
        categorias_final[categoria_mais_frequente].append(palavra)

    # Separar palavras compostas em palavras individuais
    categorias_final_final = {}
    for categoria, palavras in categorias_final.items():
        palavras_separadas = {palavra for item in palavras for palavra in item.split() if not palavra in string.punctuation and not palavra in nltk.corpus.stopwords.words('portuguese')}
        categorias_final_final[categoria] = list(palavras_separadas)

    return categorias_final_final

In [23]:
ner_entities = get_ner_entities(all_sentences)
print(ner_entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

{'data': ['1604', '1567', 'princípios', '20', '1587', '1582', 'março', '1578', '1603', '1590', 'novembro', 'janeiro', '1593', 'páscoa', 'abril'], 'profissao': ['colono', 'caçador', 'sultão', 'rainha', 'doge', 'cavalheiro', 'selvagem', 'governador', 'tabelião', 'pescador', 'vigário', 'duque', 'senhora', 'senhor', 'el', 'cacique', 'mestre', 'carmelita', 'assassino', 'capitão', 'provincial', 'escudeiro', 'soberano', 'geral', 'castelã', 'guerreiro', 'prior', 'mártir', 'prisioneiro', 'rei', 'dama', 'capitães', 'general', 'gibão', 'chefe', 'aventureiro', 'frade', 'marinheiro', 'marquês', 'índio', 'padre', 'avis', 'fidalgo', 'italiano'], 'organizacao': ['santa', 'transpontina', 'convento', 'senhora', 'ordem', 'monte', 'carmelo', 'maria', 'carmo'], 'pessoa': ['sousa', 'alcácer', 'garcia', '11', 'di', 'eva', 'batista', 'ribeiro', 'lauriana', 'salema', 'sá', 'ceci', 'diogo', 'ferreira', 'botelho', 'cunha', 'aires', 'cecília', 'coutinho', 'bernardim', 'uivando', 'belzebu', 'francisco', 'soeiro', 

### Definindo eventos com POS

In [24]:
event_verbs_tokens = [token for token in all_tokens if token.pos_ == "VERB" and not token.is_stop]
print(event_verbs_tokens)

[desliza, dirige, engrossado, recebe, torna-se, saltando, enroscando-se, espreguiçar, embeber, rola, Dir-se-ia, curva-se, Perde, revoltam, resvalam, sofre, visto, lança-se, atravessa, espumando, deixando, enchendo, falta-lhe, foge-lhe, recua, concentrar, precipita-se, fatigado, estende, adormece, formou, recebe, ostentava, virgens, estendiam, corria, formados, decorado, acabamos, descrever, fundado, havia, tivera, penetrar, via-se, construída, protegida, cortada, formava, teria, quadradas, havia, feita, Descendo, encontrava-se, construída, abria, Continuando, descer, chegava-se, curvava, sombreado, angelins, cresciam, aproveitado, criar, seguiam, alargando, fechar, tornava, edificada, apresentam, tinha, quadradas, dava, cercado, coberta, estendia-se, abria, havia, chamaremos, abraçava, copadas, fingindo, formando, criado, olhando, precipitava, tinha, parecia, feito, esmerara, criar, separado, tomado, serviam, acostados, via-se, nascido, desciam, privava, entrar, descrito, passar, podem

In [25]:
event_verbs_tokens_lemma = [format_lemma(event_verbs_token.lemma_) for event_verbs_token in event_verbs_tokens]
print(event_verbs_tokens_lemma)

['deslizar', 'dirigir', 'engrossar', 'receber', 'tornar', 'saltar', 'enroscandor', 'espreguiçar', 'embeber', 'rolar', 'dir', 'curva', 'perde', 'revoltar', 'resvalar', 'sofrer', 'ver', 'lançar', 'atravessar', 'espumar', 'deixar', 'encher', 'falta', 'foge', 'recuar', 'concentrar', 'precipita', 'fatigar', 'estender', 'adormecer', 'formar', 'receber', 'ostentar', 'virgem', 'estender', 'correr', 'formar', 'decorar', 'acabar', 'descrever', 'fundar', 'haver', 'tivera', 'penetrar', 'via', 'construir', 'protegir', 'cortar', 'formar', 'ter', 'quadrar', 'haver', 'fazer', 'descer', 'encontrar', 'construir', 'abrir', 'continuando', 'descer', 'chegava', 'curvar', 'sombrear', 'angelins', 'cresciar', 'aproveitar', 'criar', 'seguir', 'alargar', 'fechar', 'tornar', 'edificar', 'apresentar', 'ter', 'quadrar', 'dar', 'cercar', 'cobrir', 'estendia', 'abrir', 'haver', 'chamarer', 'abraçar', 'copar', 'fingir', 'formar', 'criar', 'olhar', 'precipitar', 'ter', 'parecer', 'fazer', 'esmerarar', 'criar', 'separar

### Definindo tokens de interesse

In [26]:
desirable_tokens = copy.deepcopy(ner_entities)
desirable_tokens["evento"] = copy.deepcopy(event_verbs_tokens_lemma)
desirable_tokens["objeto"] = []

# Outros tokens de interesse
desirable_tokens_values = list({value for key, values in desirable_tokens.items() for value in values})
for tokens in all_sentences_tokens:
    for token in tokens:
        lemma = format_lemma(token.lemma_)
        if not lemma in desirable_tokens_values and token.pos_ in ("PROPN", "NOUN"):
            desirable_tokens["objeto"].append(lemma)
del desirable_tokens_values

desirable_tokens["objeto"] = list(set(desirable_tokens["objeto"]))
print(desirable_tokens)

{'data': ['1604', '1567', 'princípios', '20', '1587', '1582', 'março', '1578', '1603', '1590', 'novembro', 'janeiro', '1593', 'páscoa', 'abril'], 'profissao': ['colono', 'caçador', 'sultão', 'rainha', 'doge', 'cavalheiro', 'selvagem', 'governador', 'tabelião', 'pescador', 'vigário', 'duque', 'senhora', 'senhor', 'el', 'cacique', 'mestre', 'carmelita', 'assassino', 'capitão', 'provincial', 'escudeiro', 'soberano', 'geral', 'castelã', 'guerreiro', 'prior', 'mártir', 'prisioneiro', 'rei', 'dama', 'capitães', 'general', 'gibão', 'chefe', 'aventureiro', 'frade', 'marinheiro', 'marquês', 'índio', 'padre', 'avis', 'fidalgo', 'italiano'], 'organizacao': ['santa', 'transpontina', 'convento', 'senhora', 'ordem', 'monte', 'carmelo', 'maria', 'carmo'], 'pessoa': ['sousa', 'alcácer', 'garcia', '11', 'di', 'eva', 'batista', 'ribeiro', 'lauriana', 'salema', 'sá', 'ceci', 'diogo', 'ferreira', 'botelho', 'cunha', 'aires', 'cecília', 'coutinho', 'bernardim', 'uivando', 'belzebu', 'francisco', 'soeiro', 

##Criando chunks...

In [27]:
# Cria chunks de sentenças com máximo de caracteres e porcentagem de overlap entre os chunks
def chunk_sentences(sentences, sentences_tokens, max_chars, overlap):
    chunks_sentences = []
    chunks_sentences_tokens = []

    current_chunk_text = ""
    current_chunk_sentences = []
    current_chunk_sentences_tokens = []

    for sentence_text, sentence_tokens in zip(sentences, sentences_tokens):
        new_chunk_length = len(current_chunk_text) + (1 if current_chunk_text else 0) + len(sentence_text)

        # Verifica se a sentença cabe no chunk atual
        if new_chunk_length <= max_chars:
            if current_chunk_text:
                current_chunk_text += " "
            current_chunk_text += sentence_text

            current_chunk_sentences.append(sentence_text)
            current_chunk_sentences_tokens.append(sentence_tokens)

        else:
            # Fecha o chunk atual
            if current_chunk_sentences:
                chunks_sentences.append(current_chunk_sentences)
                chunks_sentences_tokens.append(current_chunk_sentences_tokens)

            # Aplica o overlap
            num_sent_to_keep = int(len(current_chunk_sentences) * overlap)
            if num_sent_to_keep > 0:
                kept_sentences = current_chunk_sentences[-num_sent_to_keep:]
                kept_tokens = current_chunk_sentences_tokens[-num_sent_to_keep:]
                current_chunk_text = " ".join(kept_sentences)
                current_chunk_sentences = kept_sentences.copy()
                current_chunk_sentences_tokens = kept_tokens.copy()
            else:
                current_chunk_text = ""
                current_chunk_sentences = []
                current_chunk_sentences_tokens = []

            # Tenta adicionar a sentença atual no novo chunk
            if current_chunk_text:
                if len(current_chunk_text) + 1 + len(sentence_text) <= max_chars:
                    current_chunk_text += " " + sentence_text
                    current_chunk_sentences.append(sentence_text)
                    current_chunk_sentences_tokens.append(sentence_tokens)
                else:
                    # A sentença é muito grande sozinha
                    # Primeiro adiciona o chunk atual com overlap
                    if current_chunk_sentences:
                        chunks_sentences.append(current_chunk_sentences)
                        chunks_sentences_tokens.append(current_chunk_sentences_tokens)

                    # Depois adiciona a sentença sozinha como novo chunk
                    chunks_sentences.append([sentence_text])
                    chunks_sentences_tokens.append([sentence_tokens])

                    current_chunk_text = ""
                    current_chunk_sentences = []
                    current_chunk_sentences_tokens = []
            else:
                # Chunk vazio, simplesmente adiciona a sentença
                current_chunk_text = sentence_text
                current_chunk_sentences = [sentence_text]
                current_chunk_sentences_tokens = [sentence_tokens]

    # Adiciona o último chunk, se houver
    if current_chunk_sentences:
        chunks_sentences.append(current_chunk_sentences)
        chunks_sentences_tokens.append(current_chunk_sentences_tokens)

    return chunks_sentences, chunks_sentences_tokens

In [28]:
# Devolve os textos formatados dos tokens de chunks/sentences
def get_tokens_text(groups_tokens):
    return [[format_lemma(token.lemma_) for token in group_tokens] for group_tokens in groups_tokens]

In [29]:
chunks_sentences, chunks_sentences_tokens = chunk_sentences(all_sentences, all_sentences_tokens, max_chars=5000, overlap=0.8)
print(chunks_sentences[:1])
print(chunks_sentences_tokens[:1])

[['De um dos cabeços da Serra dos Órgãos desliza um fio de água que se dirige para o norte, e engrossado com os mananciais que recebe no seu curso de dez léguas, torna-se rio caudal.\n', 'É o Paquequer: saltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espreguiçar na várzea e embeber no Paraíba, que rola majestosamente em seu vasto leito.\n', 'Dir-se-ia que, vassalo e tributário desse rei das águas, o pequeno rio, altivo e sobranceiro contra os rochedos, curva-se humildemente aos pés do suserano.', 'Perde então a beleza selvática; suas ondas são calmas e serenas como as de um lago, e não se revoltam contra os barcos e as canoas que resvalam sobre elas: escravo submisso, sofre o látego do senhor.\n', 'Não é neste lugar que ele deve ser visto; sim três ou quatro léguas acima de sua foz, onde é livre ainda, como o filho indómito desta pátria da liberdade.\n', 'Aí, o Paquequer lança-se rápido sobre o seu leito, e atravessa as florestas como o tapir, espumando, 

In [30]:
chunks_texts = [" ".join(chunk_sentences) for chunk_sentences in chunks_sentences]
print(chunks_texts[:1])

['De um dos cabeços da Serra dos Órgãos desliza um fio de água que se dirige para o norte, e engrossado com os mananciais que recebe no seu curso de dez léguas, torna-se rio caudal.\n É o Paquequer: saltando de cascata em cascata, enroscando-se como uma serpente, vai depois se espreguiçar na várzea e embeber no Paraíba, que rola majestosamente em seu vasto leito.\n Dir-se-ia que, vassalo e tributário desse rei das águas, o pequeno rio, altivo e sobranceiro contra os rochedos, curva-se humildemente aos pés do suserano. Perde então a beleza selvática; suas ondas são calmas e serenas como as de um lago, e não se revoltam contra os barcos e as canoas que resvalam sobre elas: escravo submisso, sofre o látego do senhor.\n Não é neste lugar que ele deve ser visto; sim três ou quatro léguas acima de sua foz, onde é livre ainda, como o filho indómito desta pátria da liberdade.\n Aí, o Paquequer lança-se rápido sobre o seu leito, e atravessa as florestas como o tapir, espumando, deixando o pelo 

In [31]:
chunks_tokens = [[token for sentences_tokens in chunk_sentences_tokens for token in sentences_tokens] for chunk_sentences_tokens in chunks_sentences_tokens]
print(chunks_tokens[:1])

[[cabeços, Serra, Órgãos, desliza, fio, água, dirige, norte, engrossado, mananciais, recebe, curso, léguas, torna-se, rio, caudal, Paquequer, saltando, cascata, cascata, enroscando-se, serpente, espreguiçar, várzea, embeber, Paraíba, rola, majestosamente, vasto, leito, Dir-se-ia, vassalo, tributário, rei, águas, pequeno, rio, altivo, sobranceiro, rochedos, curva-se, humildemente, pés, suserano, Perde, beleza, selvática, ondas, calmas, serenas, lago, revoltam, barcos, canoas, resvalam, escravo, submisso, sofre, látego, senhor, visto, léguas, acima, foz, livre, filho, indómito, pátria, liberdade, Paquequer, lança-se, rápido, leito, atravessa, florestas, tapir, espumando, deixando, esparso, pontas, rochedo, enchendo, solidão, estampido, carreira, repente, falta-lhe, espaço, foge-lhe, terra, soberbo, rio, recua, concentrar, forças, precipita-se, arremesso, tigre, presa, fatigado, esforço, supremo, estende, terra, adormece, linda, bacia, natureza, formou, recebe, leito, noiva, cortinas, tre

In [32]:
# Seleciona cada token de interesse para cada chunk
chunks_sentences_desirables = []

for sentences_tokens in chunks_sentences_tokens:
    evaluated_sentences = []

    for sentence_tokens in sentences_tokens:
        seen_tokens = collections.defaultdict(list)

        for token in sentence_tokens:
            for desirable_group in desirable_tokens:
                lemma = format_lemma(token.lemma_)
                if lemma in desirable_tokens[desirable_group] and \
                not lemma in seen_tokens[desirable_group]:
                    seen_tokens[desirable_group].append(lemma)

        evaluated_sentences.append(seen_tokens)

    chunks_sentences_desirables.append(evaluated_sentences)

print(chunks_sentences_desirables[:1])

[[defaultdict(<class 'list'>, {'objeto': ['cabeço', 'fio', 'água', 'norte', 'manancial', 'curso', 'légua'], 'local': ['serra', 'órgãos', 'rio'], 'evento': ['deslizar', 'dirigir', 'engrossar', 'receber', 'tornar']}), defaultdict(<class 'list'>, {'local': ['paquequer', 'paraíba'], 'evento': ['saltar', 'enroscandor', 'espreguiçar', 'embeber', 'rolar'], 'objeto': ['cascata', 'serpente', 'várzea', 'leito']}), defaultdict(<class 'list'>, {'evento': ['dir', 'curva'], 'objeto': ['vassalo', 'tributário', 'água', 'pequeno', 'altivo', 'rochedo', 'pé', 'suserano'], 'profissao': ['rei'], 'local': ['rio']}), defaultdict(<class 'list'>, {'evento': ['perde', 'revoltar', 'resvalar', 'escravo', 'sofrer'], 'objeto': ['beleza', 'onda', 'calmas', 'sereno', 'lago', 'barco', 'canoa', 'submisso', 'látego'], 'profissao': ['senhor']}), defaultdict(<class 'list'>, {'evento': ['ver'], 'objeto': ['légua', 'foz', 'filho', 'indómito', 'pátria', 'liberdade']}), defaultdict(<class 'list'>, {'local': ['paquequer', 'flo

## Word2Vec

In [33]:
model_params = {
    "vector_size": 300,
    "window": 2,
    "min_count": 2,
    "workers": 4,
    "sg": 1
}

In [34]:
w2v_model = gensim.models.Word2Vec(
    sentences=get_tokens_text(chunks_tokens),
    **model_params
)

In [35]:
# Verificando palavras semelhantes
pd.DataFrame(w2v_model.wv.most_similar("peri", topn=3), columns=("word", "probalility"))

,word,probalility
0,podes,0.759973
1,espera,0.756911
2,senhora,0.753190


In [36]:
termo = "peri"
limiar = 0.3

resultados = w2v_model.wv.most_similar(termo, topn=3)  # aumenta topn se necessário
filtrados = [(palavra, sim) for palavra, sim in resultados if sim >= limiar]

pd.DataFrame(filtrados, columns=("word", "probalility"))

,word,probalility
0,podes,0.759973
1,espera,0.756911
2,senhora,0.753190


In [37]:
# Criar dicionário e corpus
dictionary = gensim.corpora.Dictionary(get_tokens_text(chunks_tokens))
corpus = [dictionary.doc2bow(tokens) for tokens in get_tokens_text(chunks_tokens)]

In [38]:
# Criar TF-IDF
tfidf_model = gensim.models.TfidfModel(corpus)

In [39]:
# Função para calcular vetor médio ponderado por TF-IDF
def calculate_mean_vector(tokens, model, dictionary, tfidf_model, desirable_tokens={}, desirable_tokens_weights={}):
    bow = dictionary.doc2bow(tokens)
    tfidf_weights = dict(tfidf_model[bow])

    valid_tokens = [dictionary[id] for id in tfidf_weights if dictionary[id] in model.wv]
    if not valid_tokens:
        return None

    vectors = []
    weights = []

    used_entity_tokens = []

    for token in valid_tokens:
        # Boost de peso se token for de interesse
        weight = tfidf_weights[dictionary.token2id[token]]
        for group, desirable in desirable_tokens.items():
            if token in desirable:
                weight *= desirable_tokens_weights[group] if group in desirable_tokens_weights else 1
                used_entity_tokens.append(token)

        vectors.append(model.wv[token] * weight)
        weights.append(weight)

    vectors = np.array(vectors)
    weights = np.array(weights).reshape(-1, 1)

    weighted_mean = np.sum(vectors, axis=0) / np.sum(weights)
    return weighted_mean

In [40]:
# Função para calcular vetores médios ponderados por TF-IDF
def calculate_mean_vectors(groups_tokens, model, dictionary, tfidf_model, desirable_tokens={}, desirable_tokens_weights={}):
    vectors = []

    # Gera vetores para cada grupo de tokens
    for group_tokens_text in get_tokens_text(groups_tokens):
        vec = calculate_mean_vector(
            group_tokens_text,
            model,
            dictionary,
            tfidf_model,
            desirable_tokens=desirable_tokens,
            desirable_tokens_weights=desirable_tokens_weights
        )
        if not vec is None:
            vectors.append(vec)

    return vectors

In [41]:
# Retorna os tokens de interesse selecionados para chunkc/sentença
def get_selected_desirables(groups_desirables, selection_desirables):
    def merge_desirables(dicts: list[dict[str, list[str]]]) -> dict[str, list[str]]:
        merged = collections.defaultdict(set)

        for d in dicts:
            for key, values in d.items():
                merged[key].update(values)

        # Convert sets back to sorted lists (optional: remove `sorted()` if not needed)
        return {key: sorted(values) for key, values in merged.items()}

    groups_desirables_formatted = [
        merge_desirables(group_desirables) if isinstance(group_desirables, list) else group_desirables
        for group_desirables in groups_desirables
    ]

    groups_desirables_selected = [
        [
            token
            for tokens_group, desirable_tokens in group_desirables.items()
            for token in desirable_tokens
            if token in selection_desirables[tokens_group]
        ]
        for group_desirables in groups_desirables_formatted
    ]

    groups_desirables_scores = [
        len(group_desirables_selected)
        for group_desirables_selected in groups_desirables_selected
    ]

    return groups_desirables_selected, groups_desirables_scores

# **Prompt**


## Recebe query...

In [42]:
# query = "Como evolui a relação entre Antônio e Peri ao longo da história?"
# query = "Como é descrita a natureza brasileira ao redor da casa de D. Antônio?"
# query = "Como é representada a lealdade de Peri a D. Antônio e Cecília"
query = "O que acontece quando Peri bebe o veneno para salvar Cecília?"

## 1: Escolhe chunk

### Pré-processmento da query

In [43]:
# Remove abreviações
query = fix_abbreviations(query)

In [44]:
def preprocess_query(query):
    tokens = nltk.tokenize.word_tokenize(query.lower())
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word not in nltk.corpus.stopwords.words('portuguese')]
    return tokens

In [45]:
# Extrai tokens da query
query_tokens = preprocess_query(query)
print(query_tokens)

['acontece', 'peri', 'bebe', 'veneno', 'salvar', 'cecília']


In [46]:
# Filtra por tokens válidos encontrados no modelo
query_tokens = [token for token in query_tokens if token in w2v_model.wv]
print(query_tokens)

['peri', 'bebe', 'veneno', 'salvar', 'cecília']


In [47]:
# Entidades encontradas na query (possível interesse)
query_desirables = {group: list({token for token in tokens if token in query_tokens}) for group, tokens in desirable_tokens.items()}
print(query_desirables)

{'data': [], 'profissao': [], 'organizacao': [], 'pessoa': ['cecília', 'peri'], 'local': [], 'evento': ['salvar'], 'objeto': ['bebe', 'veneno']}


### Cálculo dos vetores

In [48]:
# Peso para cada tipo de token de interesse
desirable_tokens_weights = {
    "profissao": 1,
    "pessoa": 1,
    "data": 1,
    "organizacao": 1,
    "local": 1,
    "evento": 1,
    "objeto": 1,
}

In [49]:
# Cálculo dos vetores das chunks com base nos interesses
chunks_vectors = calculate_mean_vectors(chunks_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

In [50]:
# Cálculo do vetor da query
query_vector = calculate_mean_vector(query_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

### Busca por similaridades

In [51]:
# Cálcula similaridades entre a query e cada chunk
chunks_similarities = sklearn.metrics.pairwise.cosine_similarity([query_vector], chunks_vectors)[0]

In [52]:
# Cácula score de interesse
chunks_desirables_selected, chunks_desirables_scores = get_selected_desirables(
    chunks_sentences_desirables,
    query_desirables
)

print(chunks_desirables_scores)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 3, 2, 3, 3, 4, 4, 4, 4, 4, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 3, 2, 3, 3, 3, 3, 2, 2, 2, 3, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1, 0, 0, 0, 0, 1, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 

In [53]:
# Similaridade por chunk
query_chunks_similarity = pd.DataFrame(
    zip(chunks_desirables_scores, chunks_similarities, chunks_texts),
    columns=("desirable_score", "similarity", "chunk")
)
query_chunks_similarity.sort_values(by=['desirable_score', 'similarity'], ascending=[False, False], inplace=True)
query_chunks_similarity.reset_index(inplace=True)

In [54]:
query_chunks_similarity

,index,desirable_score,similarity,chunk
0,498,4,0.974663,e o teu sangue e o meu sangue. Teu corpo não d...
1,497,4,0.973849,Peri vai falar.\n Álvaro entrava nesse momento...
2,496,4,0.972253,Mas qual era o resultado dessa loucura? O índi...
3,495,4,0.969954,são os únicos defensores que restam à tua senh...
4,499,4,0.969547,"""\nToda a família ouvia esta narração com uma ..."
...,...,...,...,...
607,3,0,0.840804,A casa era edificada com a arquitetura simples...
608,27,0,0.838795,os cavaleiros engatilharam os arcabuzes e segu...
609,0,0,0.837696,De um dos cabeços da Serra dos Órgãos desliza ...
610,1,0,0.833895,"De repente, falta-lhe o espaço, foge-lhe a ter..."


In [55]:
print(chunks_desirables_selected[query_chunks_similarity.iloc[0]["index"]], "\n")
query_chunks_similarity.iloc[0]["chunk"]

['veneno', 'salvar', 'cecília', 'peri'] 



'e o teu sangue e o meu sangue. Teu corpo não deve servir ao banquete do inimigo. \' " Ararê disse, e tirou suas contas de frutos que deu a seu filho: estavam cheias de veneno; tinham nelas a morte.\n" Quando Peri fosse prisioneiro, bastava quebrar um fruto, e ria do vencedor que não se animaria a tocar no seu corpo.\n "Peri viu que a senhora sofria, e olhou as suas contas; teve uma ideia; a herança de Ararê podia salvar a todos.\n" Se tu deixasses fazer o que queria, quando a noite viesse não acharia um inimigo vivo; os brancos e os índios não te ofenderiam mais. "\nToda a família ouvia esta narração com uma surpresa extraordinária; compreendiam dela que havia em tudo isto uma arma terrível — o veneno; mas não podiam saber os meios de que o índio se servira ou pretendia servir-se para usar desse agente de destruição.\n — Acaba! disse Antônio; por que modo contavas então destruir o inimigo?\n— Peri envenenou a água que os brancos bebem, e o seu corpo, que devia servir ao banquete dos A

### Chunk escolhido

In [56]:
# Delimita o chunk escolhido
chosen_chunk = query_chunks_similarity.iloc[0]["index"]
print(chosen_chunk)

498


### FUNÇÃO

In [57]:
def get_chosen_chunk(query):
    # Extrai tokens da query
    query_tokens = preprocess_query(query)

    # Filtra por tokens válidos encontrados no modelo
    query_tokens = [token for token in query_tokens if token in w2v_model.wv]

    # Entidades encontradas na query (possível interesse)
    query_desirables = {group: list({token for token in tokens if token in query_tokens}) for group, tokens in desirable_tokens.items()}

    # Cálculo dos vetores das chunks com base nos interesses
    chunks_vectors = calculate_mean_vectors(chunks_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

    # Cálculo do vetor da query
    query_vector = calculate_mean_vector(query_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

    # Cálcula similaridades entre a query e cada chunk
    if query_vector is None:
        return None
    chunks_similarities = sklearn.metrics.pairwise.cosine_similarity([query_vector], chunks_vectors)[0]

    # Cácula score de interesse
    chunks_desirables_selected, chunks_desirables_scores = get_selected_desirables(
        chunks_sentences_desirables,
        query_desirables
    )

    # Similaridade por chunk
    query_chunks_similarity = pd.DataFrame(
        zip(chunks_desirables_scores, chunks_similarities, chunks_texts),
        columns=("desirable_score", "similarity", "chunk")
    )
    query_chunks_similarity.sort_values(by=['desirable_score', 'similarity'], ascending=[False, False], inplace=True)
    query_chunks_similarity.reset_index(inplace=True)

    # Delimita o chunk escolhido
    chosen_chunk = query_chunks_similarity.iloc[0]["index"]

    return chosen_chunk

## 2: Busca por similaridades dentro do chunk

### Cálculo dos vetores das sentenças

In [58]:
w2v_chunk_model = gensim.models.Word2Vec(
    sentences=get_tokens_text(chunks_sentences_tokens[chosen_chunk]),
    **model_params
)

In [59]:
# Cálculo dos vetores das sentenças com base nos interesses
chunk_sentences_vectors = calculate_mean_vectors(chunks_sentences_tokens[chosen_chunk], w2v_chunk_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

In [60]:
# Cálculo do vetor da query
query_vector = calculate_mean_vector(query_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

### Busca por similaridades

In [61]:
# Cálcula similaridades entre a query e cada chunk
sentences_similarities = sklearn.metrics.pairwise.cosine_similarity([query_vector], chunk_sentences_vectors)[0]

In [62]:
# Cácula score de interesse
sentences_desirables_selected, sentences_desirables_scores = get_selected_desirables(
    chunks_sentences_desirables[chosen_chunk],
    query_desirables
)

print(sentences_desirables_scores)

[0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 1, 0]


In [63]:
# Similaridade por sentenças
query_sentences_similarity = pd.DataFrame(
    zip(sentences_desirables_scores, sentences_similarities, chunks_sentences[chosen_chunk]),
    columns=("desirable_score", "similarity", "chunk")
)
query_sentences_similarity.sort_values(by=['desirable_score', 'similarity'], ascending=[False, False], inplace=True)
query_sentences_similarity.reset_index(inplace=True)

In [64]:
query_sentences_similarity

,index,desirable_score,similarity,chunk
0,13,2,0.257752,O plano que Peri combinara para salvar seus am...
1,5,2,0.178894,"""Peri viu que a senhora sofria, e olhou as sua..."
2,14,2,0.107091,Confiado nesse veneno que os índios conheciam ...
3,26,2,-0.013446,Mas assim era preciso; a vida de Cecília o exi...
4,19,1,0.262082,Quando o cacique vendo-o cobrir o rosto pergun...
5,3,1,0.189280,"Ararê disse, e tirou suas contas de frutos que..."
6,8,1,0.180630,compreendiam dela que havia em tudo isto uma a...
7,11,1,0.178562,"Peri envenenou a água que os brancos bebem, e ..."
8,27,1,0.167478,"O costume dos selvagens, de não matar na guerr..."
9,4,1,0.130155,"Quando Peri fosse prisioneiro, bastava quebrar..."


### Concatena as sentenças mais similares

In [65]:
# Seleciona as sentenças mais similares de acordo com limiar
sentences_similarity_threshold = 0.4

chunk_sentences_selected = query_sentences_similarity.head(int(len(query_sentences_similarity) * sentences_similarity_threshold))
chunk_sentences_selected

,index,desirable_score,similarity,chunk
0,13,2,0.257752,O plano que Peri combinara para salvar seus am...
1,5,2,0.178894,"""Peri viu que a senhora sofria, e olhou as sua..."
2,14,2,0.107091,Confiado nesse veneno que os índios conheciam ...
3,26,2,-0.013446,Mas assim era preciso; a vida de Cecília o exi...
4,19,1,0.262082,Quando o cacique vendo-o cobrir o rosto pergun...
5,3,1,0.189280,"Ararê disse, e tirou suas contas de frutos que..."
6,8,1,0.180630,compreendiam dela que havia em tudo isto uma a...
7,11,1,0.178562,"Peri envenenou a água que os brancos bebem, e ..."
8,27,1,0.167478,"O costume dos selvagens, de não matar na guerr..."
9,4,1,0.130155,"Quando Peri fosse prisioneiro, bastava quebrar..."


In [66]:
# Em ordem de aparecimento, concatena as sentenças mais similares selecionadas
answer_context = ""
for chunk_sentence in chunks_sentences[chosen_chunk]:
    if chunk_sentence in chunk_sentences_selected["chunk"].values:
        answer_context += chunk_sentence

answer_context

'Ararê disse, e tirou suas contas de frutos que deu a seu filho: estavam cheias de veneno; tinham nelas a morte.\n"Quando Peri fosse prisioneiro, bastava quebrar um fruto, e ria do vencedor que não se animaria a tocar no seu corpo.\n"Peri viu que a senhora sofria, e olhou as suas contas; teve uma ideia; a herança de Ararê podia salvar a todos.\n"compreendiam dela que havia em tudo isto uma arma terrível — o veneno; mas não podiam saber os meios de que o índio se servira ou pretendia servir-se para usar desse agente de destruição.\nPeri envenenou a água que os brancos bebem, e o seu corpo, que devia servir ao banquete dos Aimorés!\nO plano que Peri combinara para salvar seus amigos acabava de revelar-se em toda a sua abnegação sublime e com o cortejo de cenas terríveis e monstruosas que deviam acompanhar a sua realização.\nConfiado nesse veneno que os índios conheciam com o nome de curare, e cuja fabricação era um segredo de algumas tribos, Peri com a sua inteligência e dedicação descob

### FUNÇÃO

In [67]:
def get_answer_context(query, chosen_chunk):
    # Extrai tokens da query
    query_tokens = preprocess_query(query)

    # Filtra por tokens válidos encontrados no modelo
    query_tokens = [token for token in query_tokens if token in w2v_model.wv]

    # Entidades encontradas na query (possível interesse)
    query_desirables = {group: list({token for token in tokens if token in query_tokens}) for group, tokens in desirable_tokens.items()}

    # Cálculo dos vetores das sentenças com base nos interesses
    chunk_sentences_vectors = calculate_mean_vectors(chunks_sentences_tokens[chosen_chunk], w2v_chunk_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

    # Cálculo do vetor da query
    query_vector = calculate_mean_vector(query_tokens, w2v_model, dictionary, tfidf_model, query_desirables, desirable_tokens_weights)

    # Cálcula similaridades entre a query e cada chunk
    sentences_similarities = sklearn.metrics.pairwise.cosine_similarity([query_vector], chunk_sentences_vectors)[0]

    # Cácula score de interesse
    sentences_desirables_selected, sentences_desirables_scores = get_selected_desirables(
        chunks_sentences_desirables[chosen_chunk],
        query_desirables
    )

    # Similaridade por sentenças
    query_sentences_similarity = pd.DataFrame(
        zip(sentences_desirables_scores, sentences_similarities, chunks_sentences[chosen_chunk]),
        columns=("desirable_score", "similarity", "chunk")
    )
    query_sentences_similarity.sort_values(by=['desirable_score', 'similarity'], ascending=[False, False], inplace=True)
    query_sentences_similarity.reset_index(inplace=True)

    # Seleciona as sentenças mais similares de acordo com limiar
    chunk_sentences_selected = query_sentences_similarity.head(int(len(query_sentences_similarity) * sentences_similarity_threshold))

    # Em ordem de aparecimento, concatena as sentenças mais similares selecionadas
    answer_context = ""
    for chunk_sentence in chunks_sentences[chosen_chunk]:
        if chunk_sentence in chunk_sentences_selected["chunk"].values:
            answer_context += chunk_sentence

    return answer_context

## 3: Gerando resposta

### Modelo para geração de resposta

In [68]:
# Modelo para geração de resposta (summarization)
# https://huggingface.co/recogna-nlp/ptt5-base-summ
# https://huggingface.co/unicamp-dl/ptt5-base-portuguese-vocab

sum_model = transformers.T5ForConditionalGeneration.from_pretrained("recogna-nlp/ptt5-base-summ")
sum_tokenizer = transformers.T5Tokenizer.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab", model_max_length=512)

# Adiciona tokens especiais para o prompt
sum_tokenizer.add_tokens(['<nar>', '<atributo>', '<pergunta>', '<reposta>', '<tiporesposta>', '<texto>'], special_tokens=True)
sum_model.resize_token_embeddings(len(sum_tokenizer))

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Embedding(32106, 768)

### Prompt

In [69]:
# Função para gerar prompt para geração da resposta
def get_prompt(query, answer_context):
    return "<pergunta>Responda de forma detalhada: " + query + "<texto>" + answer_context

In [70]:
# Montando o prompt para o modelo
prompt = get_prompt(query, answer_context)
# prompt = get_prompt(query, query_chunks_similarity.iloc[0]["chunk"])

### Montando resposta...

In [71]:
# https://huggingface.co/benjleite/ptt5-ptbr-qa

# Gera uma reposta com o prompt utilizando o modelo
def generate_answer(prompt):
    source_encoding = sum_tokenizer(
        prompt,
        max_length=512,
        padding='max_length',
        truncation = 'only_second',
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    input_ids = source_encoding['input_ids']
    attention_mask = source_encoding['attention_mask']

    generated_ids = sum_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_return_sequences=1,
        num_beams=5,
        max_length=1024,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        use_cache=True
    )

    prediction = {
        sum_tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    answer = ''.join(prediction)

    return answer

In [72]:
# Formata a resposta
def format_generated_answer(answer):
    answer = answer.lstrip("<pad>")
    answer = answer.rstrip("</s>")
    answer = answer.strip()
    return answer

In [73]:
# Gera resposta via prompt
answer = format_generated_answer(generate_answer(prompt))
print(answer)

Peri envenenou a água que os brancos bebem, e o seu corpo, que devia servir ao banquete dos Aimorés! O plano que Peri combinara para salvar seus amigos acabava de revelar-se em toda a sua abnegação sublime e com o cortejo de cenas terríveis e monstruosas que deviam acompanhar a sua realização.


### FUNÇÃO

In [74]:
def get_answer(query, answer_context):
    # Montando o prompt para o modelo
    prompt = get_prompt(query, answer_context)

    # Gera resposta via prompt
    answer = format_generated_answer(generate_answer(prompt))

    return answer

## Resposta

In [75]:
# RESPOSTA
print(answer_context)
print(answer)

Ararê disse, e tirou suas contas de frutos que deu a seu filho: estavam cheias de veneno; tinham nelas a morte.
"Quando Peri fosse prisioneiro, bastava quebrar um fruto, e ria do vencedor que não se animaria a tocar no seu corpo.
"Peri viu que a senhora sofria, e olhou as suas contas; teve uma ideia; a herança de Ararê podia salvar a todos.
"compreendiam dela que havia em tudo isto uma arma terrível — o veneno; mas não podiam saber os meios de que o índio se servira ou pretendia servir-se para usar desse agente de destruição.
Peri envenenou a água que os brancos bebem, e o seu corpo, que devia servir ao banquete dos Aimorés!
O plano que Peri combinara para salvar seus amigos acabava de revelar-se em toda a sua abnegação sublime e com o cortejo de cenas terríveis e monstruosas que deviam acompanhar a sua realização.
Confiado nesse veneno que os índios conheciam com o nome de curare, e cuja fabricação era um segredo de algumas tribos, Peri com a sua inteligência e dedicação descobrira um

### FUNÇÃO

In [76]:
# Função principal para executar uma query e obter uma resposta
def ask(query, should_answer_sentences=True):
    # Remove abreviações
    query = fix_abbreviations(query)

    # Delimita o chunk escolhido
    chosen_chunk = get_chosen_chunk(query)

    if chosen_chunk is None:
        return "Desculpe. Impossível detectar chunk similar com o texto dado."

    # Obtém o contexto da resposta
    answer_context = get_answer_context(query, chosen_chunk)

    if not should_answer_sentences:
        # Obtém a resposta gerada
        answer = get_answer(query, answer_context)
    else:
        # Usa o contexto como resposta de trechos do livro
        answer = answer_context

    return answer

# **Interface**

In [77]:
width = 900
x_padding = 10

box_width_px = f"{width}px"
chat_width_px = f"{width - 2 * x_padding - 4}px"

x_margin_px = "5px"
y_margin_px = "10px"
x_padding_px = f"{x_padding}px"
y_padding_px = "10px"

## Componentes

In [78]:
# Componente para espaçamento horizontal
h_spacer = widgets.Box(layout=widgets.Layout(width=x_margin_px))

### Opções extras

In [79]:
# Toggle de modo
toggle_mode = widgets.ToggleButton(
    value=True,
    description="🤖 T5",
    icon="check-circle",
    layout=widgets.Layout(
        width="100px",
        height="40px",
    )
)
toggle_mode.add_class("toggle-on")

ToggleButton(value=True, description='🤖 T5', icon='check-circle', layout=Layout(height='40px', width='100px'),…

In [80]:
# Botão de limpeza
clear_button = widgets.Button(
    description="Limpar",
    button_style="warning",  # cor amarela
    icon="trash",
    layout=widgets.Layout(width="100px", height="40px")
)

In [81]:
# Caixa para opções
options_box = widgets.HBox(
    [
        toggle_mode,
        h_spacer,
        clear_button
    ],
    layout=widgets.Layout(
        justify_content="flex-end",
        width=box_width_px,
        margin=f"{y_margin_px} 0"
    )
)

### Contâiner de chat

In [82]:
# Área de exibição do chat
chat_output = widgets.Output(
    layout=widgets.Layout(
        border="1px solid #ccc",
        padding=f"{y_padding_px} {x_padding_px}",
        width="100%",
        max_width=chat_width_px,
        height="250px",
        overflow_y="auto",
        background_color="#eef6f9",
        color="#2196F3"
    )
)

### Prompt de comandos

In [83]:
# Campo de entrada (input maior)
text_input = widgets.Text(
    placeholder="Digite sua pergunta...",
    layout=widgets.Layout(
        flex="1 1 auto",
        width="auto",
        min_width="400px"
    )
)
text_input.add_class("custom-text-input")

Text(value='', layout=Layout(flex='1 1 auto', min_width='400px', width='auto'), placeholder='Digite sua pergun…

In [84]:
# Botão de envio (botão maior)
send_button = widgets.Button(
    description="Enviar",
    button_style="primary",
    icon="paper-plane",
    layout=widgets.Layout(
        width="120px",
        height="40px"
    )
)

In [85]:
# Layout com toggle entre input e botão, centralizado
input_box = widgets.HBox(
    [
        text_input,
        h_spacer,
        send_button
    ],
    layout=widgets.Layout(
        margin=f"{y_margin_px} 0",
        justify_content="center",
        align_items="center",
        width="100%",
        max_width=box_width_px
    )
)

### Interface

In [86]:
# Corpo da interface e estilos aplicados
html_body = IPython.display.HTML(f"""
<div style="margin: {y_margin_px} 0;"><br/></div>
<div id="chat-header" style="text-align:center; margin-bottom:30px">
    <img src="https://github.com/GHLara/NLP/blob/main/logo.png?raw=true"
         alt="Chat Header" style="height: 300px; width: 300px; margin: {y_margin_px} 0;">
</div>
<style>
.output_wrapper, .output {{
    background-color: #eef6f9 !important;
}}
.toggle-on > button {{
    background-color: #28a745 !important;
    color: white !important;
    font-weight: bold;
}}
.toggle-off > button {{
    background-color: #6c757d !important;
    color: white !important;
    font-weight: normal;
    font-size: 20px;
}}
.custom-text-input > input {{
    height: 40px !important;
    font-size: 16px;
}}
body {{
    background-color: #212121 !important;
    color: #2196F3 !important;
}}
</style>
""")

In [87]:
# Container vertical para centralizar tudo
container = widgets.VBox(
    [options_box, chat_output, input_box],
    layout=widgets.Layout(
        align_items="center",
        width="100%"
    )
)

## Função de processamento

### Chamadas para pergunta

In [88]:
# Mock do processamento de query
def process_query(question, should_answer_sentences):
    return ask(question, should_answer_sentences)

### Chamada para interface

In [89]:
# Executa a interface
def display_interface():
    if chat_output:
        chat_output.clear_output()
    if text_input:
        text_input.value = ""
    IPython.display.display(html_body)
    IPython.display.display(container)

## Eventos

### Evento para *toggle* de modo

In [90]:
# Atualiza visual e texto do toggle
def update_toggle(change):
    if change["new"]:
        toggle_mode.description = "🤖 T5"
        toggle_mode.icon = "check-circle"
        toggle_mode.remove_class("toggle-off")
        toggle_mode.add_class("toggle-on")
    else:
        toggle_mode.description = "📖 Original"
        toggle_mode.icon = "times-circle"
        toggle_mode.remove_class("toggle-on")
        toggle_mode.add_class("toggle-off")

# Escutador de evento
toggle_mode.observe(update_toggle, names="value")

### Evento de limpeza

In [91]:
# Limpar saída do chat e entrada
def clear_chat(_):
    chat_output.clear_output()
    text_input.value = ""

# Escutador de evento
clear_button.on_click(clear_chat)

### Evento para envio de prompt

In [92]:
# Função de envio
def send_message(_):
    pergunta = text_input.value.strip()
    if pergunta:
        with chat_output:
            timestamp = datetime.datetime.now(zoneinfo.ZoneInfo("America/Sao_Paulo")).strftime("%H:%M:%S")
            print(f"🧑 [{timestamp}]: {pergunta}")

        resposta = process_query(pergunta, not toggle_mode.value)

        with chat_output:
            timestamp = datetime.datetime.now(zoneinfo.ZoneInfo("America/Sao_Paulo")).strftime("%H:%M:%S")
            print(f"🤖 [{timestamp}]: {resposta}")

        text_input.value = ""

# Escutadores de eventos
send_button.on_click(send_message)
text_input.on_submit(send_message)

# **EXECUÇÃO**

In [93]:
display_interface()